# Process Tweets

A script to detect the most hyped (popular) bands of a festival

In [1]:
# https://www.dataquest.io/blog/python-pandas-databases/

In [15]:
import sqlite3
import pandas as pd
import string
import unicodedata
import numpy as np
from datetime import datetime, timedelta

## Load Data

In [2]:
# Setup sqlite
sqlite_file = 'hyper_live.db'

# Connect to the database sqlite file
connection = sqlite3.connect(sqlite_file)
db = connection.cursor()

In [3]:
# get list of bands from db
bands = pd.read_sql_query("SELECT * FROM Bands;", connection)
print("{} bands read".format(len(bands)))
#bands.head()

267 bands read


In [4]:
# read ALL tweets from db
all_tweets = pd.read_sql_query("SELECT * FROM TweetsRaw", connection)
print("{} tweets in db".format(len(all_tweets)))

5879 tweets in db


In [5]:
# read ONLY tweets that have to be processed
tweets_to_process = pd.read_sql_query("SELECT * FROM TweetsRaw WHERE processed IS NULL", connection)
print("{} tweets to be processed".format(len(tweets_to_process)))

0 tweets to be processed


In [6]:
# read ONLY tweets that have to be processed
#tweets_to_process = pd.read_sql_query("SELECT tr.* FROM TweetsRaw AS tr \
#                                       LEFT JOIN BandTweets AS bt ON tr.id == bt.tweetRawId \
#                                       WHERE bt.tweetRawId IS NULL", connection)
#print("{} tweets to process".format(len(tweets_to_process)))

In [50]:
# tweets_to_process = tweets_to_process.head(50)
# print("{} tweets to process (!!!REMOVE!!!)".format(len(tweets_to_process)))

## Partition per band

Look for tweets talking about bands and re-create data structure in a band-centered way

In [51]:
def extract_bands(tweet):
    """
    Function that extracts the bands from a tweet text
    Returns a list of bands
    """
    
    # init list to return
    bands_in_tweet = []
    
    # loop all bands and check of any of the written forms is present in the tweet text
    for i, b in bands.iterrows():
                
        # set different band names writing possibilities
        bandname = b['name']
        bandname_lowercase = bandname.lower()
        bandname_lowercase_no_spaces = ''.join(bandname_lowercase.split())
        bandname_lowercase_no_accents = ''.join((c for c in unicodedata.normalize('NFD', bandname_lowercase) if unicodedata.category(c) != 'Mn'))
        bandname_lowercase_no_spaces_no_accents = ''.join((c for c in unicodedata.normalize('NFD', bandname_lowercase_no_spaces) if unicodedata.category(c) != 'Mn'))

        # check if any of the forms is in the tweet text
        if any(s in tweet['tweetText'].lower() for s in [bandname_lowercase, bandname_lowercase_no_spaces, bandname_lowercase_no_accents, bandname_lowercase_no_spaces_no_accents, b['twitterName']]):
            bands_in_tweet.append({"id": b['id'], "codedName": b['codedName']})

    return bands_in_tweet

In [52]:
def band_partition(tweet):
    """
    Function that reads a single tweet info and adds into a list the tweet information partitioned by bands.
    I.e. If a tweet mentions 2 bands, it adds a list of 2 dicts with the tweet info
    """
    
    # loop all bands and add an entry to the list
    for b in tweet['bands']:
        new_band_tweets_list.append({\
                                 "tweetRawId" : tweet['id'],\
                                 "createdAt" : tweet['createdAt'],\
                                 "storedAt" : tweet['storedAt'],\
                                 "bandId" : b['id'],\
                                 "bandCodedName" : b['codedName'],\
                                 "favsCount" : tweet['favsCount'],\
                                 "rtsCount" : tweet['rtsCount'],\
                                 "language" : tweet['language'],\
                                 "userId" : tweet['userId'],\
                                 "userFriendsCount" : tweet['userFriendsCount'],\
                                 "userFollowersCount" : tweet['userFollowersCount'],\
                                 "userStatusesCount" : tweet['userStatusesCount'],\
                                 "userFavsCount" : tweet['userFavsCount'],\
                                 "userLocation" : tweet['userLocation']\
                                })
    
    # Mark TweetsRaw as processed
    db.execute("UPDATE TweetsRaw SET processed = 1 WHERE id == {}".format(tweet.id))
    connection.commit()

In [6]:
if(tweets_to_process.shape[0] > 0):

    # extract bands for each tweet
    tweets_to_process['bands'] = tweets_to_process.apply(extract_bands, axis=1)

    # construct the list of tweets per band
    new_band_tweets_list = []
    tweets_to_process.apply(band_partition, axis=1);

    # create a dataframe from the previous list
    new_band_tweets = pd.DataFrame.from_dict(new_band_tweets_list)
    print("{} band-tweets".format(new_band_tweets.shape[0]))
else:
    new_band_tweets = pd.DataFrame([])
    print("No new tweets to process")

No new tweets to process


In [7]:
# Persist extracted tweets in DB (BandTweets table)
if(new_band_tweets.shape[0] > 0):
    new_band_tweets[['tweetRawId', 'bandId']].to_sql('BandTweets', connection, if_exists='append')
    print("{} new band tweets persisted".format(new_band_tweets.shape[0]))
else:
    print("No new band tweets to persist")

No new band tweets to persist


## Explore Band-Tweet Data

In [33]:
# Construct a dataframe joining data from TweetsRaw and BandTweets
band_tweets = pd.read_sql_query("SELECT bt.bandId, \
                                        b.name AS bandName, \
                                        b.codedName AS bandCodedName, \
                                        b.headLevel AS headLevel, \
                                        b.popularity AS popularity, \
                                        tr.* \
                                 FROM BandTweets AS bt\
                                 LEFT JOIN TweetsRaw AS tr ON bt.tweetRawId == tr.id \
                                 LEFT JOIN Bands AS b ON bt.bandId == b.id", connection)
print("{} band tweets in db".format(band_tweets.shape[0]))

1691 band tweets in db


In [96]:
# Count tweets per band
# print(band_tweets.groupby('bandCodedName').size().sort_values(ascending=False))

In [97]:
# Count measures of a given band
bandCodedName = 'arcadefire'
print("Tweets about {}:  {}".format(bandCodedName,band_tweets.groupby('bandCodedName').size()[bandCodedName]))
print("Total Favs of {}: {}".format(bandCodedName,band_tweets.groupby('bandCodedName').sum()['favsCount'][bandCodedName]))
print("Total RTs of {}:  {}".format(bandCodedName,band_tweets.groupby('bandCodedName').sum()['rtsCount'][bandCodedName]))

Tweets about arcadefire:  61
Total Favs of arcadefire: 72
Total RTs of arcadefire:  40


## Hyper Score & Ranking

Create the dataframe with one row per band and all the metrics associated

In [34]:
# group and sum retweets and favs tweets by band
band_hypes = band_tweets.groupby(['bandCodedName', 'bandName', 'bandId', 'headLevel', 'popularity']).sum()

# insert a column with the number of tweets
band_hypes['tweets'] = band_tweets.groupby(['bandCodedName', 'bandName', 'bandId', 'headLevel', 'popularity']).size()

# reset the index created in the group by
band_hypes = band_hypes.reset_index()

# delete not needed columns
del band_hypes['id']
del band_hypes['userFriendsCount']
del band_hypes['userFollowersCount']
del band_hypes['userStatusesCount']
del band_hypes['userFavsCount']
del band_hypes['processed']

# add createdAt column
band_hypes['createdAt'] = datetime.now().strftime("%a %b %d %H:%M:%S +0000 %Y")

# rename and re-order columns
band_hypes = band_hypes.rename(columns={'favsCount':'favs', 'rtsCount':'retweets'})
band_hypes = band_hypes[['bandId', 'bandCodedName', 'bandName', 'headLevel', 'popularity', 'tweets', 'favs', 'retweets', 'createdAt']]

### BF-IBP (TF-IDF revisited)
Band Frequency - Inverse BAnd Popularity
- term => band
- document => all tweets
- corpus of documents => X

#### TF: Term Frequency (normalized)
Definition:
- Measures how frequently a term occurs in a document
- Measures how frequently a band is mentioned in a set of tweets

Computation:
- number of times term t appears in a document / total number of terms in the document
- number of tweets talking about that band / total number of tweets

#### IDF: Inverse Document Frequency
Definition:
- Measures how important a term
- Measures how important a band is

Computation:
- log( total number of documents / number of documents with term t in it)
- we don't have other documents so we need another way to measure how important a band is...
  - we will consider the level of headliner of the band (according to the font size used in the festival lineup poster)
  - 5 levels: 1 (top line-up) to 5 (bottom line-up)

In [35]:
# Compute BF-IBP (Band Frequency - Inverse Band Popularity)
bf_numerator = band_hypes['tweets']*(1 + band_hypes['favs'] + band_hypes['retweets'])
band_hypes['bf_ibp'] = (bf_numerator/bf_numerator.sum()) * np.log(band_hypes['popularity'].astype(float) + 1)

### Compare Rankings

In [8]:
# Get last ranking
last_ranking = pd.read_sql_query("""
                                    SELECT * 
                                    FROM BandsHype
                                    ORDER BY bf_ibp DESC
                                """, 
                                connection)
print("LAST RANKING\n{}".format(last_ranking.head(10)))

LAST RANKING
   bandId  tweets  favs  retweets    bf_ibp  ranking_change  ranking_position
0     265      90    61        84  0.498339             0.0               1.0
1      75      38    37       254  0.424746             0.0               2.0
2      21      83    72        43  0.389408             0.0               3.0
3      66      76    56        58  0.373799             0.0               4.0
4     203      25     1       327  0.302497             0.0               5.0
5      59      30    17       211  0.254044             0.0               6.0
6     145      38    23       105  0.144739             0.0               7.0
7     158      28    14       113  0.122683             0.0               8.0
8      35      48    29        24  0.107551             0.0               9.0
9     229      24    23        83  0.100435             0.0              10.0


In [36]:
def compareBandPosition(band_row):
    """
    Function that compares the position of a band in the current ranking (in band_row) compared to the
    position in the last ranking
    """
    new_position = band_row.ranking_position
    last_postion = last_ranking['ranking_position'][last_ranking.bandId == band_row.bandId].values[0]
    return last_postion - new_position

In [37]:
# add a column indicating change in ranking
band_hypes['ranking_position'] = band_hypes['bf_ibp'].rank(ascending=0)
band_hypes['ranking_change'] = band_hypes.apply(compareBandPosition, axis=1)

### Detect trending now

In [98]:
# read the last N rankings from the historical table
last_n_rankings = pd.read_sql_query("""
                                    SELECT * 
                                    FROM BandsHypeHis
                                """, 
                                connection)

In [99]:
# create a datetime version of createdAt
last_n_rankings['createdAt_datetime'] = pd.to_datetime(last_n_rankings['createdAt'], format ='%a %b %d %H:%M:%S +0000 %Y')

# filter rankings of only last hour
last_n_rankings = last_n_rankings[ last_n_rankings['createdAt_datetime'] > (datetime.now() - timedelta(hours=1))]

In [100]:
# compute accumulated ranking changes
ranking_changes = pd.DataFrame(last_n_rankings.groupby('bandId')['ranking_change'].sum())
ranking_changes = ranking_changes.reset_index('bandId')
ranking_changes.head()

,bandId,ranking_change
0,2,0.0
1,3,0.0
2,4,0.0
3,6,0.0
4,7,0.0


In [101]:
# join dfs
band_hypes = pd.merge(band_hypes, ranking_changes, left_on='bandId', right_on='bandId', how='left')

# rename columns
del band_hypes['ranking_change_x']
band_hypes = band_hypes.rename(columns={'ranking_change_y':'trending_level'})

### Final Ranking

In [103]:
# band_hypes.sort_values(by='tweets', ascending=False).head(10)
band_hypes.sort_values(by='bf_ibp', ascending=False).head(10)

,bandId,bandCodedName,bandName,headLevel,popularity,tweets,favs,retweets,createdAt,bf_ibp,ranking_position,trending_level
162,265,tie,Tiê,5,52,90,61,84,Fri May 26 13:43:48 +0000 2017,0.498339,1.0,0.0
54,75,grandaddy,Grandaddy,3,54,38,37,254,Fri May 26 13:43:48 +0000 2017,0.424746,2.0,0.0
13,21,arcadefire,Arcade Fire,1,68,83,72,43,Fri May 26 13:43:48 +0000 2017,0.389408,3.0,0.0
45,66,frankocean,Frank Ocean,1,87,76,56,58,Fri May 26 13:43:48 +0000 2017,0.373799,4.0,0.0
149,203,teenagefanclub,Teenage Fanclub,2,46,25,1,327,Fri May 26 13:43:48 +0000 2017,0.302497,5.0,0.0
40,59,elzasoares,Elza Soares,3,47,30,17,211,Fri May 26 13:43:48 +0000 2017,0.254044,6.0,0.0
103,145,museless,Museless,4,21,38,23,105,Fri May 26 13:43:48 +0000 2017,0.144739,7.0,0.0
114,158,pavvla,PAVVLA,4,35,28,14,113,Fri May 26 13:43:48 +0000 2017,0.122683,8.0,0.0
24,35,boniver,Bon Iver,1,76,48,29,24,Fri May 26 13:43:48 +0000 2017,0.107551,9.0,0.0
161,229,thezombies,The Zombies,3,59,24,23,83,Fri May 26 13:43:48 +0000 2017,0.100435,10.0,0.0


In [104]:
# table with current ranking
band_hypes[['bandId','tweets','favs','retweets','bf_ibp', 'trending_level', 'ranking_position']].to_sql("BandsHype", connection, if_exists="replace", index=False)

# table with historical of rankings
#band_hypes[['bandId','tweets','favs','retweets','bf_ibp', 'createdAt']].to_sql("BandsHypeHist", connection_write, if_exists="append", index=False)